In [ ]:
import pandas as pd
import numpy as np
import re

# Funções utilizadas

In [ ]:
# @title Criando o dataframe
# Inicializando os vetores que serão utilizados para receber os times nas respectivas fases
top_f3 = []
top_f4 = []
campeao = []
vice = []
finalistas = []
participantes = []
reb = []
n_reb = []
ultimos = []
# Colunas do Dataframe
colunas_df = ['Time', 'LIG_A_1', 'LIG_A_2', 'LIG_A_4', 'LIG_A_8', 'LIG_A_12', 'LIG_A_NR',
              'LIG_A_16', 'LIG_A_24', 'LIG_A_32', 'LIG_A_64', 'LIG_A_100', 'LIG_A_R', 'LIG_A_ULT', 'LIG_A_P']
df_liga = pd.DataFrame(columns=colunas_df)
df_liga

,Time,LIG_A_1,LIG_A_2,LIG_A_4,LIG_A_8,LIG_A_12,LIG_A_NR,LIG_A_16,LIG_A_24,LIG_A_32,LIG_A_64,LIG_A_100,LIG_A_R,LIG_A_ULT,LIG_A_P


Dicionário de colunas

| Nome | Descrição |
|------|-----------|
| LIG_A_1 | Vencedor da edição |
| LIG_A_2 | Vice da Edição |
| LIG_A_4 | 3º e 4º lugar |
| LIG_A_8 | 5º ao 8º |
| LIG_A_16 | 9º ao 16º |
| LIG_A_R | Rebaixados |
| LIG_A_P | Número de participações |

In [ ]:
# @title Função limpar nomes
# Usando regex para limpar o '(x)' no final do nome do time
def clean_text(text):
    text = pd.Series(text).replace(r'\[.*?\]', '', regex=True).replace(r'\(.*?\)', '', regex=True)
    # Remove espaços em branco no início e no final
    return text.str.strip()

In [ ]:
# @title Função get Bracket
# Esta função pega o chaveamento da copa na Wikipedia
def get_bracket(final, bracket):
  top_4 = []
  top_8 = []
  n_reb = []
  top_f1 = []
  top_f2 = []
  top_f3 = []
  top_f4 = []
  bracket = bracket.loc[2:, :]
  participantes = final
  print(f'Finalistas: {final}')

  # A partir do final do Chaveamento anda as colunas até a fase inicial para recuperar os times
  for i in range(len(bracket.columns)-1, 0, -1):

      # Pegar os times sem repetição na coluna desejada
      times = bracket.loc[:, i].dropna().unique()
      # Retira os possíveis () e []
      times = pd.Series(times).replace(r'\[.*?\]', '', regex=True).replace(r'\(.*?\)', '', regex=True)
      # Define o tipo como String
      times = times.astype(str)
      # Garante que o conteúdo não contém apenas números
      times = times[~times.str.match(r'^[0-9\s]+$')]
      times = times.to_list()
      # Limpa o conteúdo
      times = [team.strip() for team in times if isinstance(team, str) and re.match(r'^(?!\d+$)[A-Za-zÀ-ÿ0-9\s\-]+$', team)]

      # Se a quantidade de times retornada for igual a 4, quer dizer que a coluna é a semifinal
      if (len(times) == 4) & (len(top_4) == 0):
          # Recupera os times que ainda não estão na lista 'participantes'
          top_4 = [team for team in times if team not in participantes]
          print(f'Semis:  {top_4}')
          # Adiciona os times da semifinal à lista 'participantes'
          participantes = set(participantes).union(top_4)

      elif (len(times) == 8) & (len(top_8) == 0): # 8 times = coluna Quartas de final
          top_8 = [team for team in times if team not in participantes]
          print(f'Quartas: {top_8}')
          participantes = set(participantes).union(top_8)

      elif (len(times) == 16) & (len(n_reb) == 0): # 16 times = coluna Oitavas de final
          n_reb = [team for team in times if team not in participantes]
          print(f'Oitavas: {n_reb}')
          participantes = set(participantes).union(n_reb)

      elif (len(times) > 16) & (len(times) <= 32) & (len(top_f1) == 0): # Entre 16 e 32 times = coluna -1F
          top_f1 = [team for team in times if team not in participantes]
          print(f'1ª Fase: {top_f1}')
          participantes = set(participantes).union(top_f1)

      elif (len(times) > 32) & (len(times) <= 64) & (len(top_f2) == 0): # Entre 32 e 64 times = coluna -2F
          top_f2 = [team for team in times if team not in participantes]
          print(f'2ª Fase: {top_f2}')
          participantes = set(participantes).union(top_f2)

  return participantes, top_4, top_8, n_reb, top_f1, top_f2

In [ ]:
# @title Função get Table
def get_table(table, ano):
  #dftemp = dftemp.rename(columns={'Equipevde': 'Time'})
  tabela = table.rename(columns={table.columns[0]: 'Pos', table.columns[1]: 'Time'})
  #dftemp['Pos'] = dftemp['Pos'].astype(int)
  campeao = []
  vice = []
  top_4 = []
  top_8 = []
  top_12 = []
  top_16 = []
  top_24 = []
  top_32 = []
  top_64 = []
  top_100 = []
  ultimos = []
  n_reb = []
  reb = []
  for i, row in tabela.iterrows():
    pos = float(row["Pos"])
    time = clean_text(row["Time"]).to_list()
    #last_column_value = list(getattr(row, '_asdict')().values())[-1]
    last_column_value = row.iloc[table.shape[1]-1]
    # 88-89: 4 rebaixados
    # 90-91, 94-95: 2 rebaixados
    # 93: 8 rebaixados
    if pos == 1:
      campeao = time
    elif pos == 2:
      vice = time
    elif (pos <= 4):
      top_4 = top_4 + time
    elif (pos <= 8):
      top_8 = top_8 + time
    elif (pos <= 12):
      top_12 = top_12 + time
    elif 'rebaixa' in str(last_column_value).lower():  # transforma o valor em string e faz a comparação
      reb = reb + time
    elif ((ano >= 1988) & (ano != 1992) & (ano != 1996) &
      (ano != 1999) & (ano != 2000)): # Anos com rebaixamento
      if (ano <= 1989) & (pos > len(tabela)-4): # 88-89
        reb = reb + time
      elif (ano <= 1995) & (pos > len(tabela)-2): # 90-91, 94-95
        reb = reb + time
      elif (ano == 2003) & (pos > len(tabela)-2): # 2003
        print(ano)
        reb = reb + time
      elif (ano >= 1996) & (ano != 2003) & (pos > len(tabela)-4): # 96-98, 01-02, 03-hoje
        reb = reb + time
      else:
        n_reb = n_reb + time
    elif (len(reb) > 0):
        n_reb = n_reb + time
    else: # Ano sem rebaixamento
      if (pos <= 16):
        top_16 = top_16 + time
      elif (pos <= 24):
        top_24 = top_24 + time
      elif (pos <= 32):
        top_32 = top_32 + time
      elif (pos <= 64):
        top_64 = top_64 + time
      elif (pos <= 100):
        top_100 = top_100 + time
      #n_reb = n_reb + time
      qtd_ult = round(len(tabela)*0.2)
      if (pos > len(table)-qtd_ult):
        ultimos = ultimos + time

  print(f'Participantes: {len(tabela)}')
  print(f'Campeão: {campeao}')
  print(f'Vice: {vice}')
  print(f'3º e 4º: {top_4}')
  print(f'5º ao 8º: {top_8}')
  print(f'9º ao 12º: {top_12}')
  if (len(reb) > 0):
    print(f'Não Rebaixados: {n_reb}')
    print(f'Rebaixados: {reb}')
  elif (len(table) > 12):
    print(f'Top 16: {top_16}')
    if (len(top_24) > 0):
      print(f'Top 24: {top_24}')
    if (len(top_32) > 0):
      print(f'Top 32: {top_32}')
    if (len(top_64) > 0):
      print(f'Top 64: {top_64}')
    if (len(top_100) > 0):
      print(f'Top 100: {top_100}')
    print(f'Últimos 20%: {ultimos}')
  participantes = campeao + vice + top_4 + top_8 + top_12 + n_reb + reb + top_16 + top_24 + top_32 + top_64 + top_100

  return participantes, n_reb, reb, ultimos

In [ ]:
# @title Função get Table com Beautiful Soup
def get_table_bs(url, t):
  import requests
  from bs4 import BeautifulSoup

  wikiurl = url
  table_class = "wikitable sortable jquery-tablesorter"
  response = requests.get(wikiurl)
  # 200
  soup = BeautifulSoup(response.text, 'html.parser')
  tables_all = soup.find_all('table', {"class": "wikitable"})

  # <table class="wikitable sortable">
  # <tbody><tr>
  # <th>Name of Town
  # </th>
  # <th>State
  # ....

  df = pd.read_html(str(tables_all[t]))
  return df

In [ ]:
# @title Função get times das fases preliminares
# Esta função recupera os times que foram eliminados nas fases preliminares ou
# em uma fase cuja tabela tem as colunas: Equipe 1 | Resultado | Equipe 2
def get_times_preliminares(df, list_todos):
  df_pre = pd.concat([df['Equipe 1'], df['Equipe 2']], ignore_index=True)
  top = df_pre.astype(str)
  top = top.replace(r'\[.*?\]', '', regex=True).replace(r'\(.*?\)', '', regex=True)
  top = top[~top.str.match(r'^[0-9\s]+$')]
  top = top.to_list()
  top = [team.strip() for team in top
         if isinstance(team, str) and re.match(r'^(?!\d+$)[A-Za-zÀ-ÿ0-9\s\-]+$', team)
         and team not in list_todos]
  return top

In [ ]:
# @title Função preencher tabela
def preencher_tabela(times_ed, df):
  times_tabela = df['Time'].to_list()
  # Itera sobre os participantes da edição
  for pos, team in enumerate(times_ed):
      pos += 1
      # Insere o time caso seja a primeira participação
      if team not in times_tabela:
          print(f'Novo participante: {team}')
          df.loc[len(df)] = [team] + [0] * (len(df.columns) - 1)
          # df.loc[len(df)]

      if pos == 1:
          df.loc[df['Time'] == team, 'LIG_A_1'] += 1
      elif pos == 2:
          df.loc[df['Time'] == team, 'LIG_A_2'] += 1
      elif (pos <= 4):
          df.loc[df['Time'] == team, 'LIG_A_4'] += 1
      elif (pos <= 8):
          df.loc[df['Time'] == team, 'LIG_A_8'] += 1
      elif (pos <= 12):
          df.loc[df['Time'] == team, 'LIG_A_12'] += 1
      elif team in reb:
          df.loc[df['Time'] == team, 'LIG_A_R'] += 1
      elif team in n_reb:
          df.loc[df['Time'] == team, 'LIG_A_NR'] += 1
      elif (pos <= 16):
          df.loc[df['Time'] == team, 'LIG_A_16'] += 1
      elif (pos <= 24):
          df.loc[df['Time'] == team, 'LIG_A_24'] += 1
      elif (pos <= 32):
          df.loc[df['Time'] == team, 'LIG_A_32'] += 1
      elif (pos <= 64):
          df.loc[df['Time'] == team, 'LIG_A_64'] += 1
      elif (pos <= 100):
          df.loc[df['Time'] == team, 'LIG_A_100'] += 1

      if team in ultimos:
          df.loc[df['Time'] == team, 'LIG_A_ULT'] += 1

      # Adiciona a participação do time na edição
      df.loc[df['Time'] == team, 'LIG_A_P'] += 1

  return df

# 1937: 4 Times - Quadrangular


In [ ]:
df = pd.read_html('https://pt.wikipedia.org/wiki/Campeonato_Brasileiro_de_Futebol_de_1937')
ano = int(1937)
print(f'\nCampeonato de {ano}:')
t=6
participantes, n_reb, reb, ultimos = get_table(df[t], ano)
df_liga = preencher_tabela(participantes, df_liga)


Campeonato de 1937:
Participantes: 4
Campeão: ['Atlético Mineiro']
Vice: ['Fluminense']
3º e 4º: ['Rio Branco', 'Portuguesa']
5º ao 8º: []
9º ao 12º: []
Novo participante: Atlético Mineiro
Novo participante: Fluminense
Novo participante: Rio Branco
Novo participante: Portuguesa


In [ ]:
print(f'Participantes únicos: {len(df_liga)}')

Participantes únicos: 4


# 1959-1966: 16 a 22 Times - Eliminatório


In [ ]:
# 1959: t=91, 16 times
# 1960: t=51, 17 times
# 1961, 63, 65: t=10, 18 times / 20 times / 22 times
# 1962: t=23, 18 times
# 1964: t=12, 22 times
for i in range(1959, 1966+1):
  ano = str(i)
  df = pd.read_html('https://pt.wikipedia.org/wiki/Campeonato_Brasileiro_de_Futebol_de_' + ano)
  print(f'\nCampeonato de {i}:')
  if (i == 1959):
    t=91
  elif (i == 1960):
    t=51
  elif (i == 1961) | (i == 1963) | (i == 1965):
    t=10
  elif (i == 1962):
    t=23
  elif (i == 1964):
    t=12
  elif (i == 1966):
    t=11
  participantes, n_reb, reb, ultimos = get_table(df[t], int(ano))
  df_liga = preencher_tabela(participantes, df_liga)


Campeonato de 1959:
Participantes: 16
Campeão: ['Bahia']
Vice: ['Santos']
3º e 4º: ['Grêmio', 'Vasco da Gama']
5º ao 8º: ['Sport', 'Atlético Mineiro', 'Ceará', 'Rio Branco-ES']
9º ao 12º: ['Tuna Luso', 'Athletico Paranaense', 'Ferroviário-MA', 'ABC']
Top 16: ['Hercílio Luz', 'Manufatora', 'Auto Esporte-PB', 'CSA']
Últimos 20%: ['Manufatora', 'Auto Esporte-PB', 'CSA']
Novo participante: Bahia
Novo participante: Santos
Novo participante: Grêmio
Novo participante: Vasco da Gama
Novo participante: Sport
Novo participante: Ceará
Novo participante: Rio Branco-ES
Novo participante: Tuna Luso
Novo participante: Athletico Paranaense
Novo participante: Ferroviário-MA
Novo participante: ABC
Novo participante: Hercílio Luz
Novo participante: Manufatora
Novo participante: Auto Esporte-PB
Novo participante: CSA

Campeonato de 1960:
Participantes: 17
Campeão: ['Palmeiras']
Vice: ['Fortaleza']
3º e 4º: ['Fluminense', 'Santa Cruz']
5º ao 8º: ['Bahia', 'Grêmio', 'Coritiba', 'Cruzeiro']
9º ao 12º: ['Mot

In [ ]:
print(f'Participantes únicos: {len(df_liga)}')

Participantes únicos: 71


# 1967-1968 (RGP): 15 e 17 Times - Quadrangular


In [ ]:
# 1967: t=21
# 1968: t=9
for i in range(1967, 1968+1):
  ano = str(i)
  df = pd.read_html('https://pt.wikipedia.org/wiki/Campeonato_Brasileiro_de_Futebol_de_' + str(ano) + '_(Torneio_Roberto_Gomes_Pedrosa)')
  print(f'\nTorneio Roberto Gomes Pedrosa de {i}:')
  if (i == 1967):
    t=21
  else:
    t=9
  participantes, n_reb, reb, ultimos = get_table(df[t], int(ano))
  if i == 1967:
    participantes = [nome.replace('Ferroviário', 'Ferroviário-PR') for nome in participantes]

  df_liga = preencher_tabela(participantes, df_liga)


Torneio Roberto Gomes Pedrosa de 1967:
Participantes: 15
Campeão: ['Palmeiras']
Vice: ['Internacional']
3º e 4º: ['Corinthians', 'Grêmio']
5º ao 8º: ['Portuguesa', 'Santos', 'Cruzeiro', 'Atlético Mineiro']
9º ao 12º: ['Bangu', 'São Paulo', 'Flamengo', 'Vasco da Gama']
Top 16: ['Fluminense', 'Botafogo', 'Ferroviário']
Últimos 20%: ['Fluminense', 'Botafogo', 'Ferroviário']
Novo participante: Corinthians
Novo participante: Bangu
Novo participante: São Paulo

Torneio Roberto Gomes Pedrosa de 1968:
Participantes: 17
Campeão: ['Santos']
Vice: ['Internacional']
3º e 4º: ['Vasco da Gama', 'Palmeiras']
5º ao 8º: ['Corinthians', 'Grêmio', 'Atlético Mineiro', 'Cruzeiro']
9º ao 12º: ['Atlético-PR', 'São Paulo', 'Bangu', 'Fluminense']
Top 16: ['Botafogo', 'Portuguesa', 'Flamengo', 'Bahia']
Top 24: ['Náutico']
Últimos 20%: ['Flamengo', 'Bahia', 'Náutico']
Novo participante: Atlético-PR


In [ ]:
print(f'Participantes únicos: {len(df_liga)}')

Participantes únicos: 75


# 1967-1968 (TB): 21 Times - Eliminatório


In [ ]:
# 1967: t=26
# 1968: t=17
for i in range(1967, 1968+1):
  ano = str(i)
  if (i == 1967):
    t=26
    df = pd.read_html('https://pt.wikipedia.org/wiki/Campeonato_Brasileiro_de_Futebol_de_1967_(Ta%C3%A7a_Brasil)')
    df = df[t]
  else:
    df = pd.read_html('https://pt.wikipedia.org/wiki/Campeonato_Brasileiro_de_Futebol_de_1968_(Ta%C3%A7a_Brasil)', header=0)
    t=17
    df = df[17]
    df.columns = df.iloc[0]
    df = df.drop([0,1,3,5,8,10,13,16,18,30])
    df = df.reset_index(drop=True)
    df.iloc[:, 0] = df.iloc[:, 0].str.replace('°', '', regex=False)
    df.iloc[:, 0] = df.iloc[:, 0].str.replace('º', '', regex=False)
    df.iloc[:, 0] = df.iloc[:, 0].astype(int)

  print(f'\nTaça Brasil de {i}:')
  participantes, n_reb, reb, ultimos = get_table(df, int(ano))
  # if i == 1967:
  #   participantes = [nome.replace('Ferroviário', 'Ferroviário-PR') for nome in participantes]

  df_liga = preencher_tabela(participantes, df_liga)


Taça Brasil de 1967:
Participantes: 21
Campeão: ['Palmeiras']
Vice: ['Náutico']
3º e 4º: ['Grêmio', 'Cruzeiro']
5º ao 8º: ['Atlético Mineiro', 'América-CE', 'Botafogo', 'Treze']
9º ao 12º: ['Goytacaz', 'Paysandu', 'Leônico-BA', 'Rio Branco-ES']
Top 16: ['CSA', 'Ferroviário-PR', 'Goiás', 'América de Propriá']
Top 24: ['Rabello', 'ABC', 'Piauí', 'Moto Club', 'Perdigão']
Últimos 20%: ['ABC', 'Piauí', 'Moto Club', 'Perdigão']
Novo participante: América-CE
Novo participante: Treze
Novo participante: Leônico-BA
Novo participante: Goiás
Novo participante: América de Propriá
Novo participante: Piauí
Novo participante: Perdigão

Taça Brasil de 1968:
Participantes: 21
Campeão: ['Botafogo']
Vice: ['Fortaleza']
3º e 4º: ['Cruzeiro', 'Náutico']
5º ao 8º: ['Metropol', 'Bahia', 'Atlético Goianiense', 'Moto Club']
9º ao 12º: ['Goytacaz', 'Piauí', 'Grêmio', 'Operário VG']
Top 16: ['Paysandu', 'Rabello', 'CSA', 'América de Natal']
Top 24: ['Olímpico', 'Sergipe', 'Água Verde', 'Desportiva Ferroviária', 

In [ ]:
print(f'Participantes únicos: {len(df_liga)}')

Participantes únicos: 86


# 1969-1986: 17 a 80 Times - Eliminatório

In [ ]:
# 1969: t=16, 17 times          # 1979: t=38, 94 times
# 1970: t=15, 17 times          # 1981: t=20, 44 times
# 1971: t=25, 20 times          # 1982: t=21, 44 times
# 1972: t=54, 26 times          # 1984: t=31, 41 times
# 1973: t=22, 40 times          # 1985: t=59, 44 times
# 1974: t=34, 40 times          # 1986: t=47, 80 times
# 1975/1980: t=33, 42/44 times
# 1976: t=27, 54 times
# 1977/1983: t=41, 62/44 times
# 1978: t=39, 74 times
for i in range(1969, 1986+1):
  ano = i
  print(f'\nCampeonato de {ano}:')
  if ano == 1979:
      df = pd.read_html('https://pt.wikipedia.org/wiki/Campeonato_Brasileiro_de_Futebol_de_1979')
      df = df[38]
  else:
    df = pd.read_html('https://pt.wikipedia.org/wiki/Campeonato_Brasileiro_de_Futebol_de_' + str(i))

  if ano == 1969:
    t = 16
  elif ano == 1970:
      t = 15
  elif ano == 1971:
      t = 25
  elif ano == 1972:
      t = 54
  elif ano == 1973:
      t = 22
  elif ano == 1974:
      t = 34
  elif ano == 1975 or ano == 1980:
      t = 33
  elif ano == 1976:
      t = 27
  elif ano == 1977 or ano == 1983:
      t = 41
  elif ano == 1978:
      t = 39
  elif ano == 1981:
      t = 20
  elif ano == 1982:
      t = 42
  elif ano == 1984:
      t = 31
  elif ano == 1985:
      t = 59
  elif ano == 1986:
      t = 47

  if ano == 1979:
    dftemp = df
  else:
    dftemp = df[t]

  if (i >= 1972) & (i <= 1975):
    dftemp = dftemp.drop(dftemp.index[-1])
    dftemp = dftemp.droplevel(0, axis=1)

  elif (i == 1977) | (i == 1978) | (i == 1981) | (i == 1982) | (i == 1983) | (i == 1985) | (i == 1986):
    df_middle = dftemp
    if (i != 1982) & (i != 1983):
      df_middle = df_middle.droplevel(0, axis=1)

    if (i == 1983):
      df_middle.columns = df_middle.iloc[0]
      df_middle = df_middle.loc[1:, :]
      df_middle = df_middle.reset_index(drop=True)

    if (i != 1985) & (i != 1986):
      df_middle.iloc[:, 0] = df_middle.iloc[:, 0].str.extract('(\d+)').astype(int)
    dftemp = df_middle

  participantes, n_reb, reb, ultimos = get_table(dftemp, i)
  df_liga = preencher_tabela(participantes, df_liga)


Campeonato de 1969:
Participantes: 17
Campeão: ['Palmeiras']
Vice: ['Cruzeiro']
3º e 4º: ['Corinthians', 'Botafogo']
5º ao 8º: ['Internacional', 'Atlético Mineiro', 'America', 'Santos']
9º ao 12º: ['Fluminense', 'Grêmio', 'Bahia', 'Coritiba']
Top 16: ['São Paulo', 'Santa Cruz', 'Portuguesa', 'Flamengo']
Top 24: ['Vasco da Gama']
Últimos 20%: ['Portuguesa', 'Flamengo', 'Vasco da Gama']

Campeonato de 1970:
Participantes: 17
Campeão: ['Fluminense']
Vice: ['Palmeiras']
3º e 4º: ['Atlético Mineiro', 'Cruzeiro']
5º ao 8º: ['Internacional', 'Flamengo', 'Botafogo', 'Grêmio']
9º ao 12º: ['Corinthians', 'Santos', 'Bahia', 'Santa Cruz']
Top 16: ['Atlético Paranaense', 'São Paulo', 'America-RJ', 'Ponte Preta']
Top 24: ['Vasco da Gama']
Últimos 20%: ['America-RJ', 'Ponte Preta', 'Vasco da Gama']
Novo participante: Atlético Paranaense
Novo participante: America-RJ
Novo participante: Ponte Preta

Campeonato de 1971:
Participantes: 20
Campeão: ['Atlético Mineiro']
Vice: ['São Paulo']
3º e 4º: ['Bota

In [ ]:
print(f'Participantes únicos: {len(df_liga)}')

Participantes únicos: 166


# 1987: Confusão
A classificação final ficou os times do módulo verde + os campeões e vice do cruzamento com o módulo amarelo

In [ ]:
df = pd.read_html('https://pt.wikipedia.org/wiki/Campeonato_Brasileiro_de_Futebol_de_1987')
ano = int(1987)
print(f'\nCampeonato de {ano}:')

dftemp = df[0].T.reset_index(drop=True)
dftemp.columns = dftemp.iloc[0]
dftemp = dftemp.drop(0)
dftemp = dftemp.drop(2)
campeao = clean_text(dftemp['Campeão']).to_list()
vice = clean_text(dftemp['Vice-campeão']).to_list()

dftemp = df[16]
dftemp = dftemp.droplevel(0, axis=1)
dftemp = dftemp.drop(dftemp.columns[-1], axis=1)
dftemp['Pos'] = dftemp['Pos'] + 2

df2temp = df[16+32]
df2temp = df2temp.droplevel(0, axis=1)
df2temp = df2temp.rename(columns={df2temp.columns[0]: 'Pos', df2temp.columns[1]:'Time', df2temp.columns[2]: 'PG'})
finalistas = df2temp.loc[:1, :]
dftemp.loc[len(dftemp)] = finalistas.iloc[0]
dftemp.loc[len(dftemp)] = finalistas.iloc[1]
dftemp = dftemp.sort_values(by='Pos').reset_index(drop=True)

participantes, n_reb, reb, ultimos = get_table(dftemp, ano)
df_liga = preencher_tabela(participantes, df_liga)


Campeonato de 1987:
Participantes: 18
Campeão: ['Sport']
Vice: ['Guarani']
3º e 4º: ['Flamengo', 'Internacional']
5º ao 8º: ['Atlético Mineiro', 'Cruzeiro', 'Grêmio', 'São Paulo']
9º ao 12º: ['Fluminense', 'Palmeiras', 'Botafogo', 'Vasco']
Top 16: ['Bahia', 'Coritiba', 'Goiás', 'Santa Cruz']
Top 24: ['Santos', 'Corinthians']
Últimos 20%: ['Goiás', 'Santa Cruz', 'Santos', 'Corinthians']


# 1988-1992: 20 a 24 Times - Eliminatório

In [ ]:
# 1988: t=14
# 1989: t=16
# 1990: t=15
# 1991: t=11
# 1992: t=22
for i in range(1988, 1992+1):
  ano = i
  print(f'\nCampeonato de {ano}:')
  df = pd.read_html('https://pt.wikipedia.org/wiki/Campeonato_Brasileiro_de_Futebol_de_' + str(i))

  if ano == 1988:
    t=14
  elif ano == 1989:
    t=16
  elif ano == 1990:
    t=15
  elif ano == 1991:
    t=11
  elif ano == 1992:
    t=22

  dftemp = df[t]

  if (ano == 1988) | (ano == 1989):
    dftemp = dftemp.droplevel(0, axis=1)
    dftemp = dftemp.drop(dftemp.index[-1])


  participantes, n_reb, reb, ultimos = get_table(dftemp, i)
  df_liga = preencher_tabela(participantes, df_liga)


Campeonato de 1988:
Participantes: 24
Campeão: ['Bahia']
Vice: ['Internacional']
3º e 4º: ['Fluminense', 'Grêmio']
5º ao 8º: ['Vasco da Gama', 'Flamengo', 'Sport', 'Cruzeiro']
9º ao 12º: ['Portuguesa', 'Atlético Mineiro', 'São Paulo', 'Coritiba']
Não Rebaixados: ['Goiás', 'Guarani', 'Corinthians', 'Palmeiras', 'Botafogo', 'Santos', 'Atlético Paranaense', 'Vitória']
Rebaixados: ['Bangu', 'Santa Cruz', 'Criciúma', 'America']

Campeonato de 1989:
Participantes: 22
Campeão: ['Vasco da Gama']
Vice: ['São Paulo']
3º e 4º: ['Cruzeiro', 'Botafogo']
5º ao 8º: ['Palmeiras', 'Corinthians', 'Portuguesa', 'Atlético Mineiro']
9º ao 12º: ['Flamengo', 'Goiás', 'Grêmio', 'Santos']
Não Rebaixados: ['Náutico', 'Inter de Limeira', 'Fluminense', 'Internacional', 'Vitória', 'Bahia']
Rebaixados: ['Atlético Paranaense', 'Guarani', 'Sport', 'Coritiba']

Campeonato de 1990:
Participantes: 20
Campeão: ['Corinthians']
Vice: ['São Paulo']
3º e 4º: ['Grêmio', 'Bahia']
5º ao 8º: ['Atlético Mineiro', 'Santos', 'Palm

In [ ]:
print(f'Participantes únicos: {len(df_liga)}')

Participantes únicos: 167


# 1993: 32 times, 8 rebaixados, mais confusão

In [ ]:
df = pd.read_html('https://pt.wikipedia.org/wiki/Campeonato_Brasileiro_de_Futebol_de_1993')
ano = int(1993)
print(f'\nCampeonato de {ano}:')
df = df[22]
df['Rebaixado'] = df['Classificação ou rebaixamento'].str.contains('Rebaixado', na=False)

# Ordena pelo critério da coluna 'Rebaixado', colocando os não rebaixados primeiro
df = df.sort_values(by=['Rebaixado', 'Pos.'], ascending=True).reset_index(drop=True)

# Remove a coluna auxiliar 'Rebaixado' se não for mais necessária
df = df.drop(columns='Rebaixado')
df['Pos.'] = df.index + 1
participantes, n_reb, reb, ultimos = get_table(df, ano)
df_liga = preencher_tabela(participantes, df_liga)


Campeonato de 1993:
Participantes: 32
Campeão: ['Palmeiras']
Vice: ['Vitória']
3º e 4º: ['Corinthians', 'São Paulo']
5º ao 8º: ['Santos', 'Guarani', 'Flamengo', 'Remo']
9º ao 12º: ['Portuguesa', 'Paraná', 'Paysandu', 'União São João']
Não Rebaixados: ['Grêmio', 'Criciúma', 'Cruzeiro', 'Internacional', 'Náutico', 'Bragantino', 'Vasco da Gama', 'Sport', 'Fluminense', 'Bahia', 'Botafogo', 'Atlético Mineiro']
Rebaixados: ['América Mineiro', 'Ceará', 'Coritiba', 'Santa Cruz', 'Athletico Paranaense', 'Goiás', 'Fortaleza', 'Desportiva Ferroviária']
Novo participante: Paraná
Novo participante: União São João


In [ ]:
print(f'Participantes únicos: {len(df_liga)}')

Participantes únicos: 169


# 1994-1999: 22 a 26 Times - Eliminatório

In [ ]:
# 1994: t=18
# 1995: t=14
# 1996: t=11
# 1997: t=15
# 1998: t=31
# 1999: t=10
for i in range(1994, 1999+1):
  ano = i
  print(f'\nCampeonato de {ano}:')
  df = pd.read_html('https://pt.wikipedia.org/wiki/Campeonato_Brasileiro_de_Futebol_de_' + str(i))

  if ano == 1994:
    t=18
  elif ano == 1995:
    t=14
  elif ano == 1996:
    t=11
  elif ano == 1997:
    t=15
  elif ano == 1998:
    t=31
  elif ano == 1999:
    t=10

  dftemp = df[t]

  if (ano == 1994) | (ano == 1995):
    dftemp = dftemp.droplevel(0, axis=1)
    dftemp = dftemp.drop(dftemp.index[-1])


  participantes, n_reb, reb, ultimos = get_table(dftemp, i)
  df_liga = preencher_tabela(participantes, df_liga)


Campeonato de 1994:
Participantes: 24
Campeão: ['Palmeiras']
Vice: ['Corinthians']
3º e 4º: ['Guarani', 'Atlético Mineiro']
5º ao 8º: ['Botafogo', 'São Paulo', 'Bahia', 'Bragantino']
9º ao 12º: ['Santos', 'Portuguesa', 'Grêmio', 'Vasco da Gama']
Não Rebaixados: ['Sport', 'Flamengo', 'Fluminense', 'Paysandu', 'Internacional', 'Paraná', 'Vitória', 'Criciúma', 'União São João', 'Cruzeiro']
Rebaixados: ['Remo', 'Náutico']

Campeonato de 1995:
Participantes: 24
Campeão: ['Botafogo']
Vice: ['Santos']
3º e 4º: ['Cruzeiro', 'Fluminense']
5º ao 8º: ['Palmeiras', 'Bragantino', 'Atlético Mineiro', 'Goiás']
9º ao 12º: ['Internacional', 'Portuguesa', 'Juventude', 'São Paulo']
Não Rebaixados: ['Paraná', 'Corinthians', 'Grêmio', 'Criciúma', 'Bahia', 'Sport', 'Guarani', 'Vasco da Gama', 'Flamengo', 'Vitória']
Rebaixados: ['Paysandu', 'União São João']

Campeonato de 1996:
Participantes: 24
Campeão: ['Grêmio']
Vice: ['Portuguesa']
3º e 4º: ['Atlético Mineiro', 'Goiás']
5º ao 8º: ['Cruzeiro', 'Guarani'

In [ ]:
print(f'Participantes únicos: {len(df_liga)}')

Participantes únicos: 169


# 2000: Copa João Havelange 116 Times - Eliminatório


In [ ]:
# Resolver a questão do link
# Criar uma nova coluna de últimos Lugares, com a pontuação de rebaixamento
df = pd.read_html('https://en.wikipedia.org/wiki/Copa_Jo%C3%A3o_Havelange')
dftemp = df[7].loc[1:29,:]
ano = int(2000)
print(f'\nCampeonato de {ano}:')
participantes, n_reb, reb, ultimos = get_table(dftemp, ano)
df_liga = preencher_tabela(participantes, df_liga)


Campeonato de 2000:
Participantes: 29
Campeão: ['Vasco da Gama']
Vice: ['São Caetano']
3º e 4º: ['Cruzeiro', 'Grêmio']
5º ao 8º: ['Paraná', 'Palmeiras', 'Internacional', 'Sport']
9º ao 12º: ['Ponte Preta', 'Fluminense', 'Bahia', 'São Paulo']
Top 16: ['Atlético Paranaense', 'Goiás', 'Malutrom', 'Remo']
Top 24: ['Guarani', 'Santos', 'Flamengo', 'Botafogo', 'Portuguesa', 'Vitória', 'América Mineiro', 'Atlético Mineiro']
Top 32: ['Juventude', 'Gama', 'Coritiba', 'Corinthians', 'Santa Cruz']
Últimos 20%: ['Atlético Mineiro', 'Juventude', 'Gama', 'Coritiba', 'Corinthians', 'Santa Cruz']
Novo participante: São Caetano
Novo participante: Malutrom


In [ ]:
print(f'Participantes únicos: {len(df_liga)}')

Participantes únicos: 171


# 2001-2002: 28 e 26 Times - Eliminatório


In [ ]:
# 2001-02: t=8
for i in range(2001, 2003):
  ano = str(i)
  df = pd.read_html('https://pt.wikipedia.org/wiki/Campeonato_Brasileiro_de_Futebol_de_' + ano)
  print(f'\nCampeonato de {i}:')
  t=8
  participantes, n_reb, reb, ultimos = get_table(df[t], int(ano))
  df_liga = preencher_tabela(participantes, df_liga)


Campeonato de 2001:
Participantes: 28
Campeão: ['Atlético Paranaense']
Vice: ['São Caetano']
3º e 4º: ['Fluminense', 'Atlético Mineiro']
5º ao 8º: ['Grêmio', 'Ponte Preta', 'São Paulo', 'Bahia']
9º ao 12º: ['Internacional', 'Goiás', 'Vasco da Gama', 'Palmeiras']
Não Rebaixados: ['Portuguesa', 'Paraná', 'Santos', 'Vitória', 'Coritiba', 'Corinthians', 'Guarani', 'Gama', 'Cruzeiro', 'Juventude', 'Botafogo', 'Flamengo']
Rebaixados: ['Santa Cruz', 'América Mineiro', 'Botafogo-SP', 'Sport']

Campeonato de 2002:
Participantes: 26
Campeão: ['Santos']
Vice: ['Corinthians']
3º e 4º: ['Grêmio', 'Fluminense']
5º ao 8º: ['São Paulo', 'São Caetano', 'Juventude', 'Atlético Mineiro']
9º ao 12º: ['Cruzeiro', 'Vitória', 'Coritiba', 'Goiás']
Não Rebaixados: ['Ponte Preta', 'Atlético Paranaense', 'Vasco da Gama', 'Guarani', 'Figueirense', 'Flamengo', 'Bahia', 'Paysandu', 'Internacional', 'Paraná']
Rebaixados: ['Portuguesa', 'Palmeiras', 'Gama', 'Botafogo']


In [ ]:
print(f'Participantes únicos: {len(df_liga)}')

Participantes únicos: 171


# 2003-atual: entre 24 a 20 times - Pts Corridos


In [ ]:
# 2003-04 e 2006, 2010-11: t=3
# 2007-08 e 2012: t=4
# 2009 e 2014-23: t=5
# 2013 e 2020: t=6
# 2005: t=8
for i in range(2003, 2024+1):
  ano = str(i)
  df = pd.read_html('https://pt.wikipedia.org/wiki/Campeonato_Brasileiro_de_Futebol_de_' + ano)
  print(f'\nCampeonato de {i}:')
  t=5
  if (i <= 2004) | (i == 2006) | (i == 2010) | (i == 2011):
    t = 3
  elif (i == 2007) | (i == 2008) | (i == 2012):
    t = 4
  elif (i == 2013) | (i == 2020) | (i == 2021):
    t = 6
  elif (i == 2005):
    t = 8
  participantes, n_reb, reb, ultimos = get_table(df[t], int(ano))
  df_liga = preencher_tabela(participantes, df_liga)


Campeonato de 2003:
Participantes: 24
Campeão: ['Cruzeiro']
Vice: ['Santos']
3º e 4º: ['São Paulo', 'São Caetano']
5º ao 8º: ['Coritiba', 'Internacional', 'Atlético Mineiro', 'Flamengo']
9º ao 12º: ['Goiás', 'Paraná', 'Figueirense', 'Atlético Paranaense']
Não Rebaixados: ['Guarani', 'Criciúma', 'Corinthians', 'Vitória', 'Vasco da Gama', 'Juventude', 'Fluminense', 'Grêmio', 'Ponte Preta', 'Paysandu']
Rebaixados: ['Fortaleza', 'Bahia']

Campeonato de 2004:
Participantes: 24
Campeão: ['Santos']
Vice: ['Atlético Paranaense']
3º e 4º: ['São Paulo', 'Palmeiras']
5º ao 8º: ['Corinthians', 'Goiás', 'Juventude', 'Internacional']
9º ao 12º: ['Fluminense', 'Ponte Preta', 'Figueirense', 'Coritiba']
Não Rebaixados: ['Cruzeiro', 'Paysandu', 'Paraná', 'Vasco da Gama', 'Flamengo', 'São Caetano', 'Atlético Mineiro', 'Botafogo']
Rebaixados: ['Criciúma', 'Guarani', 'Vitória', 'Grêmio']

Campeonato de 2005:
Participantes: 22
Campeão: ['Corinthians']
Vice: ['Internacional']
3º e 4º: ['Goiás', 'Palmeiras']

In [ ]:
print(f'Participantes únicos: {len(df_liga)}')

Participantes únicos: 177


# Corrigindo nomes dos times

1. Atlético Paranaense -> Athletico Paranaense
2. Atlético-PR -> Athletico Paranaense
3. América-RJ -> America-RJ
4. America -> America-RJ
5. Campo Grande -> Campo Grande-RJ
6. Cascavel Esporte Clube -> Cascavel EC
7. Comercial -> Comercial-SP
8. Grêmio Prudente -> Grêmio Barueri
6. Leônico -> Leônico-BA
2. Juventus -> Juventus-SP
8. Nacional -> Nacional-AM
9. Olímpico -> Olímpico-SC
5. Pinheiros -> Pinheiros-PR
12. Palmeiras1 -> Palmeiras
4. Rio Negro -> Rio Negro-AM
6. Rio Branco -> Rio Branco-ES
11. Rível -> River-PI
12. Tiradentes -> Tiradentes-PI
13. Vasco -> Vasco da Gama

In [ ]:
print('Times diferentes: ', len(df_liga['Time'].sort_values().to_list()))
df_liga['Time'].sort_values().to_list()

Times diferentes:  177


['ABC',
 'ASA',
 'Alecrim',
 'America',
 'America-RJ',
 'Americano',
 'América Mineiro',
 'América de Natal',
 'América de Propriá',
 'América-CE',
 'América-SP',
 'Anapolina',
 'Anápolis',
 'Athletico Paranaense',
 'Atlético Goianiense',
 'Atlético Mineiro',
 'Atlético Paranaense',
 'Atlético-PR',
 'Auto Esporte-PB',
 'Auto Esporte-PI',
 'Avaí',
 'Bahia',
 'Bangu',
 'Botafogo',
 'Botafogo-PB',
 'Botafogo-SP',
 'Bragantino',
 'Brasil de Pelotas',
 'Brasiliense',
 'Brasília',
 'CEUB',
 'CRB',
 'CSA',
 'Caldense',
 'Campinense',
 'Campo Grande',
 'Campo Grande-RJ',
 'Capelense',
 'Cascavel Esporte Clube',
 'Catuense',
 'Caxias',
 'Ceará',
 'Central',
 'Chapecoense',
 'Colatina',
 'Colorado',
 'Comercial',
 'Comercial-MS',
 'Comercial-PR',
 'Confiança',
 'Corinthians',
 'Coritiba',
 'Corumbaense',
 'Criciúma',
 'Cruzeiro',
 'Cruzeiro do Sul-DF',
 'Cuiabá',
 'Defelê',
 'Desportiva Ferroviária',
 'Dom Bosco',
 'Eletrovapo-RJ',
 'Estrela do Mar',
 'Fast Clube',
 'Ferroviária',
 'Ferroviário'

In [ ]:
substituicoes = {
    "Atlético Paranaense": "Athletico Paranaense",
    "Atlético-PR": "Athletico Paranaense",
    "América-RJ": "America-RJ",
    "America": "America-RJ",
    "Bragantino": "Red Bull Bragantino",
    "Campo Grande": "Campo Grande-RJ",
    "Cascavel Esporte Clube": "Cascavel EC",
    "Comercial": "Comercial-SP",
    "Grêmio Prudente": "Grêmio Barueri",
    "Leônico": "Leônico-BA",
    "Juventus": "Juventus-SP",
    "Nacional": "Nacional-AM",
    "Olímpico": "Olímpico-SC",
    "Pinheiros": "Pinheiros-PR",
    "Palmeiras1": "Palmeiras",
    "Rio Negro": "Rio Negro-AM",
    "Rio Branco": "Rio Branco-ES",
    "Ríver": "River-PI",
    "Tiradentes": "Tiradentes-PI",
    "Vasco": "Vasco da Gama"
}
df_liga['Time'] = df_liga['Time'].replace(substituicoes)
df_grouped = df_liga.groupby('Time', as_index=False)[colunas_df[1:]].sum()
len(df_grouped['Time'].sort_values().to_list())

162

In [ ]:
print('Total de times diferentes: ', len(df_grouped['Time'].sort_values().to_list()))
df_grouped['Time'].sort_values().to_list()

Total de times diferentes:  162


['ABC',
 'ASA',
 'Alecrim',
 'America-RJ',
 'Americano',
 'América Mineiro',
 'América de Natal',
 'América de Propriá',
 'América-CE',
 'América-SP',
 'Anapolina',
 'Anápolis',
 'Athletico Paranaense',
 'Atlético Goianiense',
 'Atlético Mineiro',
 'Auto Esporte-PB',
 'Auto Esporte-PI',
 'Avaí',
 'Bahia',
 'Bangu',
 'Botafogo',
 'Botafogo-PB',
 'Botafogo-SP',
 'Brasil de Pelotas',
 'Brasiliense',
 'Brasília',
 'CEUB',
 'CRB',
 'CSA',
 'Caldense',
 'Campinense',
 'Campo Grande-RJ',
 'Capelense',
 'Cascavel EC',
 'Catuense',
 'Caxias',
 'Ceará',
 'Central',
 'Chapecoense',
 'Colatina',
 'Colorado',
 'Comercial-MS',
 'Comercial-PR',
 'Comercial-SP',
 'Confiança',
 'Corinthians',
 'Coritiba',
 'Corumbaense',
 'Criciúma',
 'Cruzeiro',
 'Cruzeiro do Sul-DF',
 'Cuiabá',
 'Defelê',
 'Desportiva Ferroviária',
 'Dom Bosco',
 'Eletrovapo-RJ',
 'Estrela do Mar',
 'Fast Clube',
 'Ferroviária',
 'Ferroviário',
 'Ferroviário-MA',
 'Ferroviário-PR',
 'Figueirense',
 'Flamengo',
 'Flamengo-PI',
 'Flumi

In [ ]:
df_grouped.head()

,Time,LIG_A_1,LIG_A_2,LIG_A_4,LIG_A_8,LIG_A_12,LIG_A_NR,LIG_A_16,LIG_A_24,LIG_A_32,LIG_A_64,LIG_A_100,LIG_A_R,LIG_A_ULT,LIG_A_P
0,ABC,0,0,0,0,2,0,1,5,2,4,0,0,7,14
1,ASA,0,0,0,0,0,0,0,0,0,1,0,0,0,1
2,Alecrim,0,0,0,0,0,0,0,2,0,1,0,0,2,3
3,America-RJ,0,0,2,2,3,0,3,5,2,1,0,1,2,19
4,Americano,0,0,0,0,1,0,0,0,3,5,0,0,1,9


# Pontuação

Olhar para os campeonatos que não tiveram rebaixamento:
Eliminado/Últimos

| Resultado | Pontos  | Pct %   | Resultado     | Pontos   | Pct %   |
|-----------|---------|---------|---------------|----------|---------|
| SEM REBAIXAMENTO |      |     | COM REBAIXAMENTO  |         |      |
| 1º Lugar  | 2400.00 | 100.00% | 1º Lugar      |  2400.00 | 100.00% |
| 2º Lugar  |  800.00 |  33.33% | 2º Lugar      |   800.00 |  33.33% |
| 3º e 4º   |  570.00 |  23.75% | 3º e 4º       |   570.00 |  23.75% |
| 5º ao 8º  |  420.00 |  17.50% | 5º ao 8º      |   420.00 |  17.50% |
| 9º ao 12º |  300.00 |  12.50% | 9º ao 12º     |   300.00 |  12.50% |
| Top 16    |  200.00 |   8.33% | Não Rebaixado |   200.00 |   8.33% |
| Top 24    |  120.00 |   5.00% | Rebaixado     |   120.00 |   5.00% |
| Top 32    |  100.00 |   4.16% |
| Top 64    |   50.00 |   2.08% |
| Top 100   |   32.00 |   1.33% |
| Últimos 20% | Apenas Registro |   0.00% |







In [ ]:
df_rank = df_grouped.sort_values(by=colunas_df[1:], ascending=False).reset_index(drop=True)
df_rank.index += 1

In [ ]:
df_rank.head(50)

,Time,LIG_A_1,LIG_A_2,LIG_A_4,LIG_A_8,LIG_A_12,LIG_A_NR,LIG_A_16,LIG_A_24,LIG_A_32,LIG_A_64,LIG_A_100,LIG_A_R,LIG_A_ULT,LIG_A_P
1,Palmeiras,12,5,9,15,11,3,1,2,1,0,0,2,0,61
2,Santos,8,8,4,16,14,3,0,8,1,0,0,1,2,63
3,Corinthians,7,3,10,14,9,7,2,1,2,0,0,1,2,56
4,Flamengo,7,3,6,13,14,8,6,2,0,0,0,0,2,59
5,São Paulo,6,6,8,12,16,3,2,3,1,0,0,0,0,57
6,Cruzeiro,4,5,9,18,9,8,1,4,1,1,0,1,1,61
7,Vasco da Gama,4,4,4,9,17,7,2,4,0,0,0,4,2,55
8,Fluminense,4,1,12,7,8,14,4,6,1,1,0,1,2,59
9,Internacional,3,8,10,14,10,6,1,4,1,0,0,1,0,58
10,Atlético Mineiro,3,5,13,20,7,5,3,4,0,1,0,1,1,62


In [ ]:
# Pontuação em relação apenas ao Campeonato Brasileiro
titulo = 100
df_rank['PTS'] = round(df_rank['LIG_A_1']*titulo +
                  df_rank['LIG_A_2']*titulo/3 +
                  df_rank['LIG_A_4']*0.2375*titulo +
                  df_rank['LIG_A_8']*0.1750*titulo +
                  df_rank['LIG_A_12']*0.1250*titulo +
                  df_rank['LIG_A_NR']*0.0800*titulo +
                  df_rank['LIG_A_R']*0.0500*titulo +
                  df_rank['LIG_A_16']*0.0800*titulo +
                  df_rank['LIG_A_24']*0.0500*titulo +
                  df_rank['LIG_A_32']*0.0400*titulo +
                  df_rank['LIG_A_64']*0.0200*titulo +
                  df_rank['LIG_A_100']*0.0100*titulo
, 2)

In [ ]:
df_rank = df_rank.sort_values(by=['PTS', 'Time'], ascending=[False, True]).reset_index(drop=True)
df_rank.index += 1

# Ranking Geral

In [ ]:
df_rank

,Time,LIG_A_1,LIG_A_2,LIG_A_4,LIG_A_8,LIG_A_12,LIG_A_NR,LIG_A_16,LIG_A_24,LIG_A_32,LIG_A_64,LIG_A_100,LIG_A_R,LIG_A_ULT,LIG_A_P,PTS
1,Palmeiras,12,5,9,15,11,3,1,2,1,0,0,2,0,61,2036.42
2,Santos,8,8,4,16,14,3,0,8,1,0,0,1,2,63,1689.67
3,Corinthians,7,3,10,14,9,7,2,1,2,0,0,1,2,56,1485.00
4,Flamengo,7,3,6,13,14,8,6,2,0,0,0,0,2,59,1467.00
5,São Paulo,6,6,8,12,16,3,2,3,1,0,0,0,0,57,1459.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158,Guará,0,0,0,0,0,0,0,0,0,0,1,0,1,1,1.00
159,Operário-PR,0,0,0,0,0,0,0,0,0,0,1,0,1,1,1.00
160,Potiguar de Mossoró,0,0,0,0,0,0,0,0,0,0,1,0,1,1,1.00
161,Sport Belém,0,0,0,0,0,0,0,0,0,0,1,0,1,1,1.00


# Ranking Top20

In [ ]:
df_rank.loc[13:25]

,Time,LIG_A_1,LIG_A_2,LIG_A_4,LIG_A_8,LIG_A_12,LIG_A_NR,LIG_A_16,LIG_A_24,LIG_A_32,LIG_A_64,LIG_A_100,LIG_A_R,LIG_A_ULT,LIG_A_P,PTS
13,Bahia,2,2,1,8,10,9,5,6,3,1,0,4,3,51,731.42
14,Athletico Paranaense,1,1,2,9,11,9,3,1,4,2,0,4,1,47,616.83
15,Coritiba,1,0,2,7,9,7,3,3,1,2,0,7,2,42,520.50
16,Sport,1,0,1,9,8,8,0,3,5,1,0,6,2,42,512.25
17,Guarani,1,2,2,3,4,7,5,1,1,0,0,3,0,29,436.67
18,Goiás,0,0,2,7,8,5,6,5,2,2,0,6,2,43,425.00
19,Vitória,0,1,1,5,6,11,1,3,4,2,0,5,0,39,375.58
20,Portuguesa,0,1,2,4,8,3,3,7,3,1,0,3,5,35,362.83
21,Náutico,0,1,4,2,1,6,5,4,3,4,0,3,3,33,318.83
22,Fortaleza,0,2,2,4,2,2,2,2,2,4,1,3,3,26,283.17


In [ ]:
# Exportar para Excel
df_rank.to_excel('Campeonato Brasileiro Rank.xlsx')